# Generative Spaces (ABM)

In this workshop we will learn how to construct a ABM (Agent Based Model) with spatial behaviours, that is capable of configuring the space. This file is a simplified version of Generative Spatial Agent Based Models. For further information, you can find more advanced versions here:

* [Object Oriented version](https://github.com/shervinazadi/spatial_computing_workshops/blob/master/notebooks/w3_generative_spaces.ipynb)
* [Vectorized version](https://topogenesis.readthedocs.io/notebooks/random_walker)

## 0. Initialization

### 0.1. Load required libraries

In [1]:
# !pip install pyvista==0.28.1 ipyvtklink

In [2]:
import os
import topogenesis as tg
import pyvista as pv
import trimesh as tm
import pandas as pd
import numpy as np

np.random.seed(0)

### 0.2. Define the Neighborhood (Stencil)

In [3]:
# creating neighborhood definition
stencil = tg.create_stencil("von_neumann", 1, 1)
# setting the center to zero
stencil.set_index([0,0,0], 0)
print(stencil)

[[[0 0 0]
  [0 1 0]
  [0 0 0]]

 [[0 1 0]
  [1 0 1]
  [0 1 0]]

 [[0 0 0]
  [0 1 0]
  [0 0 0]]]


### 0.3 Visualize the Stencil

In [4]:
# initiating the plotter
p = pv.Plotter(notebook=True)

# Create the spatial reference
grid = pv.UniformGrid()

# Set the grid dimensions: shape because we want to inject our values
grid.dimensions = np.array(stencil.shape) + 1
# The bottom left corner of the data set
grid.origin = [0,0,0]
# These are the cell sizes along each axis
grid.spacing = [1,1,1]

# Add the data values to the cell data
grid.cell_arrays["values"] = stencil.flatten(order="F")  # Flatten the stencil
threshed = grid.threshold([0.9, 1.1])

# adding the voxels: light red
p.add_mesh(threshed, show_edges=True, color="#ff8fa3", opacity=0.3)

# plotting
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(7.295554957734411, 7.295554957734411, 7.295554957734411),
 (1.5, 1.5, 1.5),
 (0.0, 0.0, 1.0)]

## 1. Setup the Environment

### 1.1. Load the envelope lattice as the avialbility lattice

In [5]:
# loading the lattice from csv
lattice_path = os.path.relpath('../data/voxelized_envelope.csv')
avail_lattice = tg.lattice_from_csv(lattice_path)
init_avail_lattice = tg.to_lattice(np.copy(avail_lattice), avail_lattice)

### 1.2 Load Program

In [6]:
# 加载偏好值
program_complete = pd.read_csv("../data/program_small.csv")
program_complete

,space_name,space_id,str_acc,sun_acc
0,lobby,0,1.0,0.0
1,roof_garden,1,0.0,1.0
2,workshop,2,0.5,0.5


In [7]:
# 进一步简化
program_prefs = program_complete.drop(["space_name","space_id"], 1)
program_prefs

,str_acc,sun_acc
0,1.0,0.0
1,0.0,1.0
2,0.5,0.5


### 1.2 Load the value fields

In [8]:
# loading the lattice from csv 只是一个看起来有些麻烦的读取罢了
fields = {}
for f in program_prefs.columns:
    lattice_path = os.path.relpath('../data/' + f + '.csv') # 因为我们的命名方式是这样的
    fields[f] = tg.lattice_from_csv(lattice_path)

### 1.3. Initialize the Agents

In [9]:
# initialize the occupation lattice
occ_lattice = avail_lattice * 0 - 1 # 把所有设置成-1就是都可以用

# Finding the index of the available voxels in avail_lattice
avail_flat = avail_lattice.flatten()
avail_index = np.array(np.where(avail_lattice)).T # 因为avail lattice中外面那一圈是不可用的，之后要定位到？

# Randomly choosing three available voxels
agn_num = len(program_complete) 
select_id = np.random.choice(len(avail_index), agn_num)
agn_origins = avail_index[select_id] # 选择的流程是，在availindex的长度中抽一个数，对应到avail index上得到坐标

# adding the origins to the agents locations
agn_locs = []
# for each agent origin ... 
for a_id, a_origin in enumerate(agn_origins):

    # add the origin to the list of agent locations
    agn_locs.append([a_origin])

    # set the origin in availablity lattice as 0 (UNavailable)
    avail_lattice[tuple(a_origin)] = 0 # 不管是什么，都不available了

    # set the origin in occupation lattice as the agent id (a_id)
    occ_lattice[tuple(a_origin)] = a_id # 在这里标记是什么agent占用了3D中哪一个

### 1.4. Visualize the environment

In [10]:
p = pv.Plotter(notebook=True)

# Set the grid dimensions: shape + 1 because we want to inject our values on the CELL data
grid = pv.UniformGrid()
grid.dimensions = np.array(occ_lattice.shape) + 1
# The bottom left corner of the data set
grid.origin = occ_lattice.minbound - occ_lattice.unit * 0.5
# These are the cell sizes along each axis
grid.spacing = occ_lattice.unit 

# adding the boundingbox wireframe
p.add_mesh(grid.outline(), color="grey", label="Domain")

# adding axes
p.add_axes()
p.show_bounds(grid="back", location="back", color="#777777")

# Add the data values to the cell data
grid.cell_arrays["Agents"] = occ_lattice.flatten(order="F").astype(int)  # Flatten the array!
# filtering the voxels
threshed = grid.threshold([-0.1, agn_num - 0.9])
# adding the voxels
p.add_mesh(threshed, show_edges=True, opacity=1.0, show_scalar_bar=False)

# adding the availability lattice
init_avail_lattice.fast_vis(p)

p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(247.21077512227743, 152.21077512227743, 232.21077512227743),
 (35.0, -60.0, 20.0),
 (0.0, 0.0, 1.0)]

## 2. ABM Simulation (Agent Based Space Occupation)

### 2.1. Running the simulation

In [11]:
program_complete

,space_name,space_id,str_acc,sun_acc
0,lobby,0,1.0,0.0
1,roof_garden,1,0.0,1.0
2,workshop,2,0.5,0.5


In [12]:
program_test = pd.read_excel("../data/program_test.xlsx")

In [13]:
# To create the second requirement, I will write a method to calculate euclidian distance in lattice
# This part uses the same idea in w2 distance_fields
def distance_field(a_id):
    lattice_cens = init_avail_lattice.centroids_threshold(-1)
    dist_m = []
    for voxel_cen in lattice_cens:
        dist_v = []
        for agent_location in agn_locs[a_id]:
            diff = voxel_cen - agent_location
            diff_p2 = diff**2
            diff_p2s = diff_p2.sum()
            dist = diff_p2s**0.5
            dist_v.append(dist)
        dist_m.append(dist_v)
    dist_m = np.array(dist_m)
    min_dist = dist_m.min(axis=1)
    agent_eu_distance_lattice = tg.to_lattice(min_dist.reshape(init_avail_lattice.shape), init_avail_lattice)
    envelope_eu_dist_lattice = agent_eu_distance_lattice * init_avail_lattice
    return(envelope_eu_dist_lattice)

# initialize fields
for a_id, a_prefs in program_complete.iterrows():
    fields[a_id] = distance_field(a_id)
        

In [15]:
# make a deep copy of occupation lattice
cur_occ_lattice = tg.to_lattice(np.copy(occ_lattice), occ_lattice)
# initialzing the list of frames
frames = [cur_occ_lattice]

# setting the time variable to 0
t = 0
n_frames = 30
# main feedback loop of the simulation (for each time step ...)
while t<n_frames: # 在每一步的时候
    # for each agent ... 对每一个row
    for a_id, a_prefs in program_complete.iterrows():
        # retrieve the list of the locations of the current agent
        a_locs = agn_locs[a_id] # program complete里第几个就是agent location中第几个，之前随机出来的地方
        # initialize the list of free neighbours
        free_neighs = []
        # for each location of the agent 现在每个agent占的每一个点都找free neighbor
        for loc in a_locs:
            # retrieve the list of neighbours of the agent based on the stencil
            neighs = avail_lattice.find_neighbours_masked(stencil, loc = loc) # 对每个点找neighbor
            # for each neighbour ... 
            for n in neighs:
                # compute 3D index of neighbour
                neigh_3d_id = np.unravel_index(n, avail_lattice.shape) #找3d的index，目的是为了看有没有available
                # if the neighbour is available... 
                if avail_lattice[neigh_3d_id]: # True就说明是free的
                    # add the neighbour to the list of free neighbours
                    free_neighs.append(neigh_3d_id) # free的话就加进去，但是不怕加重复的吗！！！！
        # check if found any free neighbour 现在已经看过全部neighbor了
        if len(free_neighs)>0:
            # convert free neighbours to a numpy array
            fns = np.array(free_neighs)

            # find the value of neighbours
            # init the agent value array
            a_eval = np.ones(len(fns)) # 先把值设定为1，之后再算每一个点哪个好
            # 任何关于选哪个的，都要从这里下手
            # for each field...
            for f in program_prefs.columns: # 这里的f只是名字罢了
                # find the raw value of free neighbours...
                vals = fields[f][fns[:,0], fns[:,1], fns[:,2]] # vals得到的应该是长度为fns长度的，在对应voxel地点的f值
                # raise the the raw value to the power of preference weight of the agent
                a_weighted_vals = vals ** a_prefs[f] # 对值做power，这个aperfs来源于之前最开头定是哪个agent
                # multiply them to the previous weighted values
                a_eval *= a_weighted_vals # 得到最终结果
            
            # now this part add the desirable space vicinity into the account
            for s in program_test.columns:
                vals = fields[s][fns[:,0], fns[:,1], fns[:,2]]
                a_weighted_vals = vals ** program_test[a_id][s]
                a_eval *= a_weighted_vals

            # This is the part that takes square shape of room into the account
            square_weight = 0.1
            free_neighs_count = []
            for free_neigh in free_neighs:
                free_neighs_count.append(free_neighs.count(free_neigh))
            # 0.1 can be modified, I choose it to make it less effective
            a_weighted_square = np.array(free_neighs_count) ** square_weight
            a_eval *= a_weighted_square

            # Here, when it reaches the maximum space needed, start to evaluate inner voxels
            current_length = np.copy(len(a_locs))
            i_eval = np.zeros(current_length)
            # 20 can be modified
            max_space = 15
            if current_length >= max_space:
                i_eval = np.ones(current_length)
                for f in program_prefs.columns:
                    vals = fields[f][np.array(a_locs)[:,0], np.array(a_locs)[:,1], np.array(a_locs)[:,2]]
                    a_weighted_vals = vals ** a_prefs[f]
                    i_eval *= a_weighted_vals

                for s in program_test.columns:
                    vals = fields[s][np.array(a_locs)[:,0], np.array(a_locs)[:,1], np.array(a_locs)[:,2]]
                    a_weighted_vals = vals ** program_test[a_id][s]
                    i_eval *= a_weighted_vals
                
                i_neighs_count = np.zeros(current_length)
                for id,loc in enumerate(a_locs):
                    neighs = init_avail_lattice.find_neighbours_masked(stencil, loc = loc)
                    for n in neighs:
                        neigh_3d_id = np.unravel_index(n, avail_lattice.shape)
                        i_neighs_count[id] += (occ_lattice==a_id)[neigh_3d_id]
                i_weighted_square = np.array(i_neighs_count) ** square_weight
                i_eval *= i_weighted_square

            # select the inner with lowest evaluation
            selected_int_inner = np.argmin(i_eval)
            # select the neighbour with highest evaluation
            selected_int = np.argmax(a_eval)

            if (current_length >= max_space) and i_eval[selected_int_inner] < a_eval[selected_int]:
                selected_inner_3d_id = tuple(a_locs[selected_int_inner])
                selected_inner_loc = a_locs[selected_int_inner]
                # print(selected_inner_3d_id)
                agn_locs[a_id].pop(selected_int_inner)
                avail_lattice[selected_inner_3d_id] = 1
                occ_lattice[selected_inner_3d_id] = -1

            if (not current_length >= max_space) or (current_length >= max_space and i_eval[selected_int_inner] < a_eval[selected_int]):
                # find 3D integer index of selected neighbour
                selected_neigh_3d_id = free_neighs[selected_int] # free neighs对应的总是一样的
                # find the location of the newly selected neighbour
                selected_neigh_loc = np.array(selected_neigh_3d_id).flatten()
                # add the newly selected neighbour location to agent locations
                agn_locs[a_id].append(selected_neigh_loc)
                # set the newly selected neighbour as UNavailable (0) in the availability lattice
                avail_lattice[selected_neigh_3d_id] = 0 # 变成不可用
                # set the newly selected neighbour as OCCUPIED by current agent 
                # (-1 means not-occupied so a_id)
                occ_lattice[selected_neigh_3d_id] = a_id # occupation lattice变成对应agent的序号
            
            # Here distance fields are updated (after agn_locs)
            fields[a_id] = distance_field(a_id)

    # constructing the new lattice
    new_occ_lattice = tg.to_lattice(np.copy(occ_lattice), occ_lattice)
    # adding the new lattice to the list of frames
    frames.append(new_occ_lattice) # 这个是用于之后画图可以随t变化
    # adding one to the time counter
    t += 1

### 2.2. Visualizing the simulation

In [16]:
p = pv.Plotter(notebook=True)

base_lattice = frames[0]

# Set the grid dimensions: shape + 1 because we want to inject our values on the CELL data
grid = pv.UniformGrid()
grid.dimensions = np.array(base_lattice.shape) + 1
# The bottom left corner of the data set
grid.origin = base_lattice.minbound - base_lattice.unit * 0.5
# These are the cell sizes along each axis
grid.spacing = base_lattice.unit 

# adding the boundingbox wireframe
p.add_mesh(grid.outline(), color="grey", label="Domain")

# adding the availability lattice
init_avail_lattice.fast_vis(p)

# adding axes
p.add_axes()
p.show_bounds(grid="back", location="back", color="#aaaaaa")

def create_mesh(value):
    f = int(value)
    lattice = frames[f]

    # Add the data values to the cell data
    grid.cell_arrays["Agents"] = lattice.flatten(order="F").astype(int)  # Flatten the array!
    # filtering the voxels
    threshed = grid.threshold([-0.1, agn_num - 0.9])
    # adding the voxels
    p.add_mesh(threshed, name='sphere', show_edges=True, opacity=1.0, show_scalar_bar=False)

    return

p.add_slider_widget(create_mesh, [0, n_frames], title='Time', value=0, event_type="always", style="classic")
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(247.21077512227743, 152.21077512227743, 232.21077512227743),
 (35.0, -60.0, 20.0),
 (0.0, 0.0, 1.0)]

### 2.3. Saving lattice frames in CSV

In [17]:
for i, lattice in enumerate(frames):
    csv_path = os.path.relpath('../data/abm_animation/abm_f_'+ f'{i:03}' + '.csv')
    lattice.to_csv(csv_path)

### Credits

In [18]:
__author__ = "Shervin Azadi "
__license__ = "MIT"
__version__ = "1.0"
__url__ = "https://github.com/shervinazadi/spatial_computing_workshops"
__summary__ = "Spatial Computing Design Studio Workshop on Agent Based Models for Generative Spaces"